# File-io Homework

In this week's homework you will try to familiarize yourself with reading in data from FITS files. Some of these problems will deal with flux data, others with spectra from galaxies and/or stars.

In [ ]:
from astropy.io import fits
import numpy as np
import pylab as plt
from scipy.signal import medfilt
from scipy.interpolate import interp1d

# Review:

This section will help with helping you recollect the main things to take away from FITS files and how to navigate them. 

# #1

a) What is stored in a FITS file. List as many things as you can come up with.

b) How does one access data from a FITS file.

c) Where is all the information about whats in the FITS file located at and how can you use this to access crucial information?

d) Do most FITS file have one thing inside the file or multiple? How can you check this?

a)

b)

c)

d)

# #2

In [ ]:
#below are a list of code with an empty comment above the line. Your task is 
#to describe what each line is doing please be as specific as you can. 


#
x = fits.open('Spectrum.fits')
    
#
print(x.info())
    
#
a = x[0].header

#
print(repr(a))

#
b = x[1].header
       
#
print(list(b.keys()))
    
#
data = x[1].data['SPEC']
    
#
data1 = x[1].data['WAVELN']

#
x.close()

# #3

In this problem you will open a file and give back some useful information from just the header and info.

In [ ]:
hdu = fits.open('dss.14.4.4.98432000000392+0.59.53.3939999999998.fits')

#How many things are in the FITS file? Please print out the names below





#What are the dimensions of each thing?





#Please write code that extracts the data from PrimaryHDU





#Please write code that extracts the columns from the BinTableHDU, what are their 
#names and units associated with them





A big push in astronomy nowadays is that big surveys go out and collect a ton of data. How much data do they collect? To put things in perspective Aaron Parson, the radio astronomy professor here has a project called HERA and he's looking at HI lines from the very first galaxies ever formed. He mentioned to us that in one night of observing he collects something like 250 Terabytes of data. That is a ton of data!!!! You can only imagine that other projects like the SDSS, Kepler, TESS, and the most anticipated James Webb Space Telescope, collect or will collect a similar amount. Thus, knowing how to handle data in any format is a crucial tool for an astronomer to have. You will be using these big data releases to answer any questions that you want to answer. Which you will explore in this weeks homework. The goal of this homework is to familiarize yourself with file-io particularly how to grab data from the files and how to use the data in these files to do some cool astronomy.

# 1 Data from Fits

In this first homework problem you will attempt to grab data from a website and access the neccessary data that we ask for. 

1. 
Download the fits binary table from the FIRST survey from this website, http://sundog.stsci.edu/first/catalogs/readme.html, labeled first_14dec17.fits.gz.

Write a function that:

    a. Opens the file and prints out the RA and DEC of objects that have a peak flux of 100 mJy or more. 

    b. Reads in the fits file and calculates the total flux recieved from the southern hemisphere (ie, below the equator). NOTE: This is not the same as using FPEAK as in part a.

# 2 Working with Spectrum Data

Working in astronomy means that our primary mode of getting information about an object is by analyzing the light we recieve from that object. Thus being familiar with how to work with spectrum data and reducing it down so that it looks nice is crucial to further our understanding of astronomical objects. 

They are important in getting chemical composition of stars by looking at absorption and emission lines.
By looking for certain lines in the spectrum and seeing how it changes from its original rest frame emission frequency or wavelength we can determine how fast the object is moving away or towards us and is a proxy for detemrining how far away the object is from us.

For this problem you will need to work with the following fits file:

    star1.fits
    star2.fits
    galaxy1.fits
    galaxy2.fits
    
Lets open one up and see whats inside.    

In [ ]:
hdu = fits.open('star1.fits')

In [ ]:
primary_hdr = hdu[0].header

In [ ]:
data_hdr = hdu[1].header
print(repr(data_hdr))

The data we want to plot are those labeled flux and loglam. Flux will be in the conventional astronomical cgs units which have units of $10^{-17}\frac{ergs}{s \space cm^2 \space Angstrom}$ and wavelength ($\lambda$) will be in Angstroms

# 2a. 
Make a function that takes the spectrum data for all of the files above and plots them. Be sure to label key absorption and emission lines that astronomers look for. 

Some rest frame wavelength are:
    
    [OII] = 3727.1 & 3729.9 Angstroms
    [OIII] = 4364.4 & 4960.3 & 5008.2 Angstroma
    HeII = 4687 & 5413 Angstroms
    Ha = 6564.6 Angstroms (h-alpha)
    Hb = 4862.7 Angstroms (h-beta)
    [Ar III] = 7137.8 Angstroms

Something that we can check from the plot(s) you made in part a) is the shift of an emission line. You might notice that some lines do not seem to match up with the peaks of the lines. This has to do with the objects relative motion to us. 

If the peak seems to have shifted to lower wavelength that means the object is moving towards us and is blueshifted. If the peak seems to have shifted to higher wavelength that means the object is moving away from us and is redshifted. 

By looking at this shift in the wavelength/frequency of an emission line astronomers can determine just how fast an object is moving away or towards us. This is the doppler velocity which you will calculate in the next problem.

# 2b. 
Make a function that calculate the doppler velocity of the galaxy specturm using the doppler velocity formula and using the strongest emission line you can find in the spectrum.

The doppler velocity formula is given by: 

$ \frac{\Delta\nu}{\nu} = -\frac{v}{c}$

Where $\Delta\nu$ is the change in frequency from rest frame frequency to the frequency where a line is observed at. Need not be too exact but it does have to be reasonable.

Hint: c = $\lambda \nu$ relates wavelength to frequency

# 3 Data Reduction of a Spectrum

Now we will apply what we learned from the first two problems and apply them to some radio astronomy data. The files that you will use were taken from the RadioLab course. What we set out to do is obtain spectra of the Milky Way Galaxy by measuring the HI emission line emitting from the spiral arms. 

We used the telescope called Leuschner which lies in the Berkeley hills to collect the data. There we used a spectrometer (a fancy name for a device that splits up light into different wavelengths/frequencies and measures the intensity at each wavelength) to measure the intensities near the HI rest frequency line. But as is typical of all observations there are some errors coming in due to the equipment being used. Often times the signals are very weak and in order for us to see them they have to go through an amplifier so the signal can get boosted. However, this introduces noise into the signal. Another source of noise comes when we apply low pass and high pass filters; we can get a shape due to the filter that we do not want in our final signal. So this problem will teach you how to handle that and reduce the data down.

# Step 1

The first thing to do is to calibrate your incoming signal with the proper gain (a little into what gain is later). 

To do this we take a spectrum of a patch of the sky, not really pointing at anything, and then we take another spectrum looking at the same patch of the sky but now we introduce some noise.

These files are the noise_on.fits and noise_off.fits files you will be working with. 

We will see just how the intrumentation set-up introduces a baseline shape. 

In [ ]:
#open up the two fits files and lets extract one sample of the signal and plot it

#Insert code here:
hdu_on = fits.open('noise_on.fits') #on data
hdu_off = fits.open('noise_off.fits') #off data

#Lets look at the header for the on data
#to see what information we can get and what would be of importance for this problem

hdr = 

print(repr(hdr))

---

Lets unpack the header as there is alot of information here:

NSPEC: represents the number of specta that we have acquired for this pointing in this case it is 100 spectra

SAMPRATE: is the rate at which we sampled the light coming into our spectrometer

BW: is the Bandwidth of our Spectra (this is used to create our x-axis frequency array)

NCHAN: represents the number of frequency channel or in laymans terms just how big spectra is (try using len(data) to see) (this information is also crucial to making our x-axis array)

RA and DEC: are the coordinates that the telescope was pointing at

L and B are the galactic coordinate the telescope was looking at

INTTIME: was how long each specra was integrated over in this case about .7 seconds per spectrum

In [ ]:
#lets look at the information in the hdu_on
hdu_on.info()

This fits files gives us 100 spectrums starting at 1-100

---

Lets look at the header of our first data BinTable

In [ ]:
#fill in 
data_hdr = hdu_on[1].header

print(repr(data_hdr))

# NOTE: 
The Leuschner telescope is able to observe an object in two different polarizations
auto0_real is vertical and auto1_real is horizontal polarization for this problem we restrict ourself to auto0_real

In [ ]:
#we will now extract the first data in the fits file: 
spec_on = hdu_on[1].data['auto0_real']

#now do the same for the off data
spec_off = 

#Now lets plot this spectrum just by itself to get a feel for the data
#It would be wise to plot them on the same figure to see the difference between noise on and noise off

plt.figure(figsize = (14,8))

plt.legend()
plt.show()

So what we will do is clean it up by taking the average over all the spectra that we have.

## 3a. 
Write a function that is able to find the average of the 100 spectra in the fits file and apply your function to the on and off fits files and plot the results on the same figure. 

Recall your prior knowledge of arrays and its operations to help you with this problem.

In [ ]:
#write code below


#making the x-axis array for you to use in the plot:
bw = hdu_on[0].header['BW']/1e6
n = hdu_on[0].header['nchan']
#This is the frequency array and has units of MHz be sure to label this in your plot
xaxis = np.linspace(-bw/2, bw/2, n) + (2*635) + 150 

#plotting code below

plt.plot(xaxis, )

Once you have plotted the average of 100 spectra for the on and off data, hopefully you got a spectra that looks a lot cleaner and is not as noisy as before. 

# Step 2

Next in the data reduction is to calculate something called the Gain with your averaged on and off data. The Gain is a number that we multiply our signal and is essentially a calibration. So to calibrate the signal we use the gain formula given below:

G = $\frac{T_{sys, on} - T_{sys,off}}{\sum (s_{on} - s_{off})} \sum s_{off}$ 

Here $T_{sys,on}$ is the temperature of the telescope when the noise is turned on, which in our case is 30K, and $T_{sys,off}$ is the temperature of the telescope when the noise is turned off, which is the ambient space temperature of 2.73 or 3 K for simplicity. $s_{on}$ and $s_{off}$ are the avgeraged spectra you calculated for the signal when noise is on and off respectively. 

## 3b. 
Calculate the gain of the Leuschner telscope using the formula above

# Step 3

We are almost done in our data reduction. Now that we know what to multiply the signal to properly calibrate it all we have to do is get a good signal multiply it by the gain and we are done. However, we run into the problem that the bandpass shape from our filter is altering our signal. So we need a way to get rid of those pesky sine waves and parabolic shape from the signal.

Use the function that you used in part a) of this problem to smooth out the cassiopea signal and plot it. This can be found in the cassiopea.fits file.  

In [ ]:

#label for your smoothed cassiopea spectrum
cass_avg = 

To get rid of the huge spikes we see popping up randomly in the data we can further filter it using scipy's medfilt function

In [ ]:
smoothed_cass = medfilt(cass_avg, kernel_size = 5)

The signal we are trying to get is those two peaks in the middle of the data. 

To get rid of the bandpass shape we will be using interpolation. As mentioned in class, interpolation returns another function where it guesses values in between missing data points with a polynomial that you specify. I highly recommend using scipy's interp1d function set kind = quadratic or cubic.

## 3c. 
Use any interpolation method to get rid of the baseline shape (ie, the sine and parabolic shape) in the cassiopea data and plot the result.

In [ ]:
#this code block creates a boolean filter to get rid of the signal we are trying to preserve
#you use this to filter the data and use the filtered data in the interpolation functions
bool_filter = np.ones(len(smoothed_cass), dtype = bool)

for i in range(len(smoothed_cass)):
    if xaxis[i] >= 1420 and xaxis[i] <= 1421.5:
        bool_filter[i] = False

interp_data = smoothed_cass[bool_filter]      